<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Connect and analyze Open Table Format data across clouds and catalogs
  <br>
       <img id="teradata-logo" src="../../images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>

<img src="images/What_is_OTF.png" style="width:600px">
<div style="position:absolute;top:120px;left:700px;color:#00233C">
    <p>Brings data warehouse capabilities for managing data to the data lake
        <li>Table abstraction</li>
        <li>Open metadata specification</li>
        <li>ACID for concurrent reads/writes</li>
        <li>Table evolution : Schema evolution, Partition evolution, Sort order evolution
        <li>Time travel</li>
        <li>Support multiple file formats</li>
    </p>
<div>

In [2]:
%addconnect name=otf_demo, host=XXX.XXX.XXX.XXX

Success: 'tf_demo' connection added

In [4]:
%connect otf_demo, user=data_engineer, hidewarnings=True

Password: ········


Success: 'otf_demo' connection established and activated for user 'data_engineer'


<h3 style = 'color:#00233C'>Establish connectivity to <strong style = 'color:#009FDB'>AWS Glue</strong> data lake </h3>

In [4]:
--First, we'll create an authorization to establish the credendials for AWS resources
REPLACE AUTHORIZATION glue_auth_aws
AS INVOKER TRUSTED
USER 'EXAMPLE'
PASSWORD 'EXAMPLE';

Success: 0 rows affected

In [3]:
--Second we create the connectivity to the Iceberg Glue Data Lake
REPLACE DATALAKE aws_glue_catalog
EXTERNAL SECURITY INVOKER TRUSTED CATALOG glue_auth_aws,
EXTERNAL SECURITY INVOKER TRUSTED STORAGE glue_auth_aws
USING
storage_location ('s3://samplebucket/glue-db')
catalog_type ('glue')
storage_region ('us-west-2')
TABLE FORMAT iceberg;

Success: 9 rows affected

<h3 style = 'color:#00233C'>Establish connectivity to <strong style = 'color:#009FDB'>Azure Unity</strong> data lake </h3>

In [6]:
--First, we'll create an authorization to establish the credendials for Azure resources
REPLACE AUTHORIZATION unity_auth_azure
AS INVOKER TRUSTED 
-- Azure AD service principal client id
USER 'EXAMPLE'
-- Azure AD service principal client secret key
PASSWORD 'EXAMPLE';

Success: 0 rows affected

In [4]:
--Second we create the connectivity to the Iceberg Unity Data Lake
REPLACE DATALAKE azure_unity_catalog
EXTERNAL SECURITY INVOKER TRUSTED CATALOG unity_auth_azure,
EXTERNAL SECURITY INVOKER TRUSTED STORAGE unity_auth_azure
USING
catalog_type ('unity')
catalog_location ('https://samplelocation.azuredatabricks.net/api/2.1/unity-catalog/iceberg')
unity_catalog_name ('demo_catalog')
storage_account_name ('sampleaccount')
tenant_id ('sampletenant')
TABLE FORMAT iceberg;

Success: 11 rows affected

<h3 style = ';color:#00233C'> Now let's confirm our OTF Iceberg Data Lakes </h3>

In [5]:
SELECT servername, createtimestamp, authorizationname, tableformat 
FROM dbc.serverv
WHERE ServerName like '%aws%' OR
      Servername like '%azure%';

,ServerName,CreateTimeStamp,AuthorizationName,TableFormat
1,azure_unity_catalog,2024-06-10 15:19:03,UNITY_AUTH_AZURE,ICEBERG
2,aws_glue_catalog,2024-04-05 17:38:41,GLUE_AUTH_AWS,ICEBERG
3,aws_glue_catalog_ng,2024-07-11 02:55:17,GLUE_AUTH_AWS_NG,ICEBERG


<h2 style = 'font-size:28px;font-family:Arial;color:#00233C'>Let's discover what is available in each of our Iceberg Data Lakes</h2><br>
<img src="images/VCL_Iceberg_Data_Lakes.png" style="width:500px">
<div style="position:absolute;top:60px;left:560px">
    <p>1) Inspect what databases are available in each catalog</p>
    <p>2) Retrieve the tables are available in the database</p>
    <p>3) Inspect the columns in the table(s)</p>
    <p>4) Explore the metadata content available in the catalog (snapshots, manifests, history, partitions)</p>
    <p>5) We will query data from the Iceberg Data Lake</p>
    <p>6) We will join data across clouds and catalogs</p>
    <div>

<h3 style = 'color:#00233C'>Show all databases in each catalog</h3>

In [ ]:
HELP DATALAKE aws_glue_catalog;

In [7]:
HELP DATALAKE azure_unity_catalog;

,DatabaseName,DatabaseProperties
1,default,
2,information_schema,
3,retail_schema,


<h3 style = 'color:#00233C'>Show all tables in AWS Glue <strong style = 'color:#009FDB'>aws_glue_catalog.tddemos_glue_db</strong> database </h3>

In [8]:
HELP DATABASE aws_glue_catalog.tddemos_glue_db;

,TableName,TableProperties
1,analytic_data_set,"'teradata.purge.all'='true','write.object-storage.enabled'='true','write.parquet.compression-codec'='zstd'"
2,customer_journey,"'teradata.purge.all'='true','write.object-storage.enabled'='true','write.parquet.compression-codec'='zstd'"
3,glove_6b_50d,"'teradata.purge.all'='true','write.object-storage.enabled'='true','write.parquet.compression-codec'='zstd'"
4,retail_events,"'write.format.default'='PARQUET','write.object-storage.enabled'='true','write.parquet.compression-codec'='ZSTD'"
5,uk_retail_data,"'teradata.purge.all'='true','write.object-storage.enabled'='true','write.parquet.compression-codec'='zstd'"
6,web_comment,"'teradata.purge.all'='true','write.object-storage.enabled'='true','write.parquet.compression-codec'='zstd'"


<h3 style = 'color:#00233C'>Show all columns in AWS Glue <strong style = 'color:#009FDB'>aws_glue_catalog.tddemos_glue_db.retail_events</strong> table </h3>

In [9]:
HELP TABLE aws_glue_catalog.tddemos_glue_db.retail_events;

,ColumnName,IcebergType,TeradataType
1,customer_id,int,INTEGER_DT
2,datestamp,timestamp,TIMESTAMP_DT
3,event,string,VARCHAR_DT(32) CHARACTER SET UNICODE


<img src="images/Apache Iceberg OTF Logical Architecture.png" style="width:800px">

<h3 style = ';color:#00233C'>Explore time travel available in the AWS Glue <strong style = 'color:#727272'>aws_glue_catalog.tddemos_glue_db.retail_events</strong> table </h3>

In [ ]:
--Show snapshots for AWS Glue datalake_iceberg_glue.tddemos_glue_db.retail_events table
SELECT * FROM TD_SNAPSHOTS(ON (aws_glue_catalog.tddemos_glue_db.retail_events)) D;

In [11]:
--Show time travel using snapshots for AWS Glue datalake_iceberg_glue.tddemos_glue_db.retail_events table
SELECT TOP 10 * from aws_glue_catalog.tddemos_glue_db.retail_events FOR SNAPSHOT AS OF '4481719215509051254';

,customer_id,datestamp,event
1,113169,2018-04-11 11:45:00.000000,Web Chat
2,105623,2018-04-08 13:27:00.000000,Return Policy In
3,224320,2018-04-08 07:43:00.000000,Product Browsing
4,28886,2018-05-15 11:59:00.000000,Mem Purchase
5,24596,2018-04-17 00:25:00.000000,Neutral Call
6,32596,2018-04-08 21:03:00.000000,Purchase
7,112739,2018-05-13 15:22:00.000000,Product Browsing
8,47794,2018-03-21 03:44:00.000000,Neutral Call
9,65776,2018-03-19 12:22:00.000000,Neutral Call
10,52873,2018-05-08 11:55:00.000000,Web Chat


<h3 style = ';color:#00233C'>Run a simple query from <strong style = 'color:#727272'>AWS Glue</strong> Catalog</h3></h3>

In [12]:
--let's select with some qualifying columns and filters
SELECT customer_id, datestamp, event FROM aws_glue_catalog.tddemos_glue_db.retail_events WHERE event='Web Chat' AND customer_id=52873;

,customer_id,datestamp,event
1,52873,2018-05-08 11:55:00.000000,Web Chat


<h3 style = 'color:#00233C'>Run a simple query data from <strong style = 'color:#727272'>Azure Unity</strong> Catalog</h3>

In [13]:
SELECT TOP 10 * FROM azure_unity_catalog.retail_schema.customer_comments;

,comment_id,customer_id,comment_text,comment_summary
1,11399,2596,"""Not what i was expecting, but pleasantly surprised. was envision",soft yet supportive
2,9319,2360,,
3,2814,2047,"Such a cute pattern, and i loved the fold-over design, but beware",Beware - not enough bottom coverage!
4,61,1894,"3 tags sewn in, 2 small (about 1'' long) and 1 huge (about 2'' x 3",Itchy tags
5,1672,2585,Can't go wrong with these new charlie's by cartonnier. they fit am,Another winner
6,7565,2475,This is my new favorite sweater. soft and comfortable and easy to,New favorite
7,7239,2432,I have a hard time with sizes here. i literally can wear sm-large,Cute but too big need to exchange
8,5628,2466,"I love this sweater. it's warm, comfortable, and soft. nice qualit",Comfortable sweater
9,13866,2394,I just received this and absolutely love it !!! i love cold should,Just adorable !! a must for every wardro
10,19046,2603,Hack: wrap the sash around waist twice and tie in back!! agreed wi,Comfortable and classy


<h3 style = 'color:#00233C'>Multi-Cloud (<strong style = 'color:#727272'>Azure</strong> & <strong style = 'color:#727272'>AWS</strong>) join of 2 Iceberg tables (<strong style = 'color:#7030A0'>Azure Unity</strong> & <strong style = 'color:#009FDB'>AWS Glue</strong>) and one <strong style = 'color:#7030A0'>Vantage</strong> dimension table  </h3>

In [14]:
SELECT TOP 10
    l.F_Name,
    l.L_Name,
    gc.event,
    gc.datestamp,
    az.comment_summary
    
FROM demo.retail_customer l --Local Dimension Table

JOIN aws_glue_catalog.tddemos_glue_db.retail_events gc -- AWS Glue Customer Events
    ON gc.customer_id = l.customer_id
    
JOIN azure_unity_catalog.retail_schema.customer_comments az -- Azure Unity Customer Comments
    ON gc.customer_id = az.customer_id
  

,F_NAME,L_NAME,event,datestamp,comment_summary
1,Jeromy,Hughes,Service Inquiry,2018-04-07 10:06:00.000000,i normally wear medium or small tops. f
2,Virgil,Montoya,Purchase,2018-03-25 17:10:00.000000,Scooped knit slip
3,Aubrey,Bryant,Purchase,2018-05-08 23:39:00.000000,Cute romper
4,Millard,Ho,Product Browsing,2018-04-29 17:22:00.000000,although comfortable
5,Dean,King,Web Chat,2018-05-01 16:06:00.000000,Great basic!
6,Kirsten,Davies,Store Visit,2018-03-14 14:34:00.000000,Skip it
7,Deann,Sellers,Store Visit,2018-05-09 20:46:00.000000,Super sexy
8,Harriett,Murillo,Product Browsing,2018-05-01 09:07:00.000000,Are you kidding?
9,Ian,Stout,Neutral Call,2018-03-23 06:44:00.000000,
10,Velma,Gill,Mem Purchase,2018-04-24 02:18:00.000000,


<hr>
<h2 style = 'font-size:28px;font-family:Arial;color:#00233C'>Write data to AWS Glue catalog</h2><br>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use <b>standard</b> SQL syntax to create and insert data into an Iceberg table in AWS Glue</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>DROP</b> Table if necessary</li>
    <li><b>CREATE TABLE AS...WITH DATA</b> to take the result of the previous query and write it to the table</li>
    <li><b>INSERT</b> additional rows</li>
    <li>Inspect snapshots</li>
    <li>Perform a simple query</li>
    </ol>
    
<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.  Drop the table</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If necessary, drop the table so we can recreate it.  PURGE ALL will remove all prior snapshots.</p>

In [86]:
DROP TABLE aws_glue_catalog.tddemos_glue_db.analytic_data_set PURGE ALL;

Success: 0 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.  Create the table</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard CREATE TABLE statement, the only modification is the multi-dot notation</p>

In [ ]:
CREATE TABLE aws_glue_catalog.tddemos_glue_db.analytic_data_set AS (
    SELECT TOP 10
        l.F_Name,
        l.L_Name,
        gc.event,
        gc.datestamp,
        az.comment_summary

    FROM demo.retail_customer l --Local Dimension Table

    JOIN aws_glue_catalog.tddemos_glue_db.retail_events gc -- AWS Glue Customer Events
        ON gc.customer_id = l.customer_id

    JOIN azure_unity_catalog.retail_schema.customer_comments az -- Azure Unity Customer Comments
        ON gc.customer_id = az.customer_id
) WITH DATA;
  

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.  Insert rows</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard INSERT statement</p>

In [15]:
INSERT INTO aws_glue_catalog.tddemos_glue_db.analytic_data_set
    SELECT TOP 10
        l.F_Name,
        l.L_Name,
        gc.event,
        gc.datestamp,
        az.comment_summary

    FROM demo.retail_customer l --Local Dimension Table

    JOIN aws_glue_catalog.tddemos_glue_db.retail_events gc -- AWS Glue Customer Events
        ON gc.customer_id = l.customer_id

    JOIN azure_unity_catalog.retail_schema.customer_comments az -- Azure Unity Customer Comments
        ON gc.customer_id = az.customer_id;

Success: 10 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4. Inspect Snapshots</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the TD_SNAPSHOTS function to view and verify the inserts.</p>

In [ ]:
--Show snapshots for the create and insert
SELECT * FROM TD_SNAPSHOTS(ON (aws_glue_catalog.tddemos_glue_db.analytic_data_set)) D;

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.  Query the table</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Simple query as above</p>

In [17]:
SELECT * FROM aws_glue_catalog.tddemos_glue_db.analytic_data_set;

,f_name,l_name,event,datestamp,comment_summary
1,Lino,Bullock,Product Browsing,2018-04-18 04:32:00.000000,Best tights!
2,Lino,Bullock,Product Browsing,2018-04-18 04:32:00.000000,Best tights!
3,Hong,Lutz,Product Browsing,2018-05-04 19:37:00.000000,5-star fantastic!
4,Angelique,Wade,Web Chat,2018-04-02 10:45:00.000000,"Cute, poorly made"
5,Melinda,Valenzue,Return Policy In,2018-05-13 13:24:00.000000,Great fit
6,Jordan,Mclean,Store Visit,2018-03-24 20:08:00.000000,Super comfy and cute
7,Angelique,Wade,Web Chat,2018-04-02 10:45:00.000000,"Cute, poorly made"
8,Melinda,Valenzue,Return Policy In,2018-05-13 13:24:00.000000,Great fit
9,Jeromy,Hughes,Purchase,2018-04-07 10:12:00.000000,i normally wear med
10,Gabriela,Osborn,Store Visit,2018-03-18 21:14:00.000000,Comfortable and flat


<h2> End of demo </h2>

In [24]:
%disconnect otf_demo

Success: 'otf_demo' disconnected